In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # don't show all the tensorflow startup messages

import numpy as np
from keras import models
from sys import path
from utils.data_generator import data_generator
from utils.test_utils import get_labels, get_indices, tuple_gen, viewer
from input import Input

In [2]:
# get predictions and convert numerical values to class name
def get_predictions(input, classes, model):
    gen = data_generator(
        f'{path[0]}/processed/{input.dset_name}_test.h5',
        'examples',
        classes,
        shuffle=False
    )

    pred = model.predict(gen, use_multiprocessing=True, workers=8)

    return np.argmax(pred, axis=1), pred

In [3]:
def test(model_name, input, classes):
    model = models.load_model(model_name)

    # predictions represented as integer representation of classes
    best, probs = get_predictions(input, classes, model)


    # labels with class and image number
    labels = get_labels(input)

    indices = get_indices(input)
    


    results = tuple_gen(labels, probs, indices)

    return results

    # viewer(results, classes, 0)

    # def update(index):
    #     viewer(results, classes, index)

    # from ipywidgets import interact, IntSlider

    # interact(update, index=IntSlider(min=0, max=len(results)-1, step=1, value=0))

In [5]:
classes = ['facebook', 'instagram', 'orig', 'telegram', 'twitter',  'whatsapp']
dataset = 'iplab'
epochs = 10
batch_size = 32
architecture = 'dct_cnn_2017'
location = 'dct_models'
h_input = Input(dataset, patch_size=64, sf=[1, 10], his_range=[-50, 50], domain="Histogram")
n_input = Input(dataset, domain="Noise", patch_size=64)
name = f'models/cnn_{architecture}_{h_input.input_shape}_{epochs}_{batch_size}'

results = test(name, h_input, classes)

def update(index):
    viewer(results, classes, index)

from ipywidgets import interact, IntSlider

interact(update, index=IntSlider(min=0, max=len(results)-1, step=1, value=0))

2357/2357 [==============================] - 5s 1ms/step


interactive(children=(IntSlider(value=0, description='index', max=143), Output()), _dom_classes=('widget-inter…

<function __main__.update(index)>

In [24]:
import numpy as np
import os
import matplotlib.pyplot as plt
from jpeg2dct.numpy import loads
from PIL import Image
import io


def hist(dct, sf, his_range):
    sf_num = len(range(*sf))
    # indexes to DC and all AC coefficients
    indexes = [
        (0,0), (0, 1), (1, 0), (2, 0), (1, 1), (0, 2), (0, 3), (1, 2),
        (2, 1), (3, 0), (4, 0), (3, 1), (2, 2),(1, 3), (0, 4), (0, 5),
        (1, 4), (2, 3), (3, 2), (4, 1), (5, 0), (6, 0), (5, 1), (4, 2),
        (3, 3), (2, 4), (1, 5), (0, 6), (0, 7), (1, 6), (2, 5), (3, 4),
        (4, 3), (5, 2), (6, 1), (7, 0), (7, 1), (6, 2), (5, 3), (4, 4),
        (3, 5), (2, 6), (1, 7), (2, 7), (3, 6), (4, 5), (5, 4), (6, 3),
        (7, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (4, 7), (5, 6),
        (6, 5), (7, 4), (7, 5), (6, 6), (5, 7), (6, 7), (7, 6), (7, 7)
    ]

    bin_num = len(range(*his_range)) + 1


    coords = indexes[sf[0]: sf[1]]

    # build histogram
    his = np.zeros((sf_num, bin_num))

    c_H = len(dct)
    c_W = len(dct[0])

    # # iterate over each 8x8 block in a patch and build up histogram
    for x in range(c_H):
        for y in range(c_W):
            sf = np.array([dct[x][y].reshape(8,8)[c[0]][c[1]] for c in coords])

            for i, f in enumerate(sf):
                h, _ = np.histogram(f, bins=bin_num, range=his_range)
                his[i] += h # update counts in histogram

    return his.flatten()


def process(patches):
    histograms = []

    for p in patches:
            # extract dct coefficients
            dct, _, _ =  loads(p)
            display(dct)

            # this is just to stop numba complaining 
            his_range = (-50, 50)
            sf = (1, 10)

            # build histograms
            histogram = hist(dct, sf, his_range)
            histograms.append(histogram)
            
    return histograms


fb = f'/home/lazy/snimageclassification/iplab//facebook/CANON_650D_720X480_INDOOR/1.jpg'
wa = f'/home/lazy/snimageclassification/iplab//whatsapp/CANON_650D_720X480_INDOOR/IMG-20160314-WA0011.jpg'
orig = '/home/lazy/snimageclassification/iplab/orig/CANON_650D_720X480_INDOOR/IMG_2635.JPG'

# convert patch to bytes so jpeg2dct can load it
def im_to_bytes(patch, q):
    buf = io.BytesIO()
    patch.save(buf, format='JPEG', qtables=q)
    return buf.getvalue()

def make_patches(image, patch_size, q=None, to_bytes=True):
    patches = []
    p = []
    for i in range(0, image.width-patch_size+1, patch_size):
        for j in range(0, image.height-patch_size+1, patch_size):
            patch = image.crop((i, j, i+patch_size, j+patch_size))
            p.append(patch)
            if to_bytes:
                patch = im_to_bytes(patch, q)

            patches.append(patch)
  
    return patches, p  

image = Image.open(fb)
 # get q tables
qtable = image.quantization

# break down image to patches
patches, p = make_patches(image, 64, qtable, True)

p_h = process(patches)








array([[[ 180,    0,    0, ...,    0,    0,    0],
        [ 189,    0,    0, ...,    0,    0,    0],
        [ 198,    0,    0, ...,    0,    0,    0],
        ...,
        [ 216,    0,    0, ...,    0,    0,    0],
        [ 216,    0,    0, ...,    0,    0,    0],
        [ 225,    0,    0, ...,    0,    0,    0]],

       [[ 207,    0,    0, ...,    0,    0,    0],
        [ 216,    0,    0, ...,    0,    0,    0],
        [ 225,    0,    0, ...,    0,    0,    0],
        ...,
        [ 243,    0,    6, ...,    0,    0,    0],
        [ 252,    0,    0, ...,    0,    0,    0],
        [ 252,    0,    0, ...,    0,    0,    0]],

       [[ 225,    0,    0, ...,    0,    0,    0],
        [ 243,    0,    0, ...,    0,    0,    0],
        [ 243,    0,    0, ...,    0,    0,    0],
        ...,
        [ 270,    0,    0, ...,    0,    0,    0],
        [ 270,    0,    0, ...,    0,    0,    0],
        [ 279,    0,    0, ...,    0,    0,    0]],

       ...,

       [[  27,  -30,    

array([[[-621,    0,    0, ...,    0,    0,    0],
        [-621,    0,    0, ...,    0,    0,    0],
        [-612,    0,    0, ...,    0,    0,    0],
        ...,
        [-612,    0,    0, ...,    0,    0,    0],
        [-603,    0,    0, ...,    0,    0,    0],
        [-603,    0,    0, ...,    0,    0,    0]],

       [[-621,    0,    0, ...,    0,    0,    0],
        [-612,    0,    0, ...,    0,    0,    0],
        [-603,    0,    0, ...,    0,    0,    0],
        ...,
        [-594,    0,    0, ...,    0,    0,    0],
        [-594,    0,    0, ...,    0,    0,    0],
        [-594,    0,    0, ...,    0,    0,    0]],

       [[-621,    0,    0, ...,    0,    0,    0],
        [-621,   -6,    0, ...,    0,    0,    0],
        [-612,    0,    0, ...,    0,    0,    0],
        ...,
        [-603,    0,    0, ...,    0,    0,    0],
        [-603,    0,    0, ...,    0,    0,    0],
        [-603,    0,    0, ...,    0,    0,    0]],

       ...,

       [[-612,    0,    

array([[[-585,    0,    0, ...,    0,    0,    0],
        [-585,    0,    0, ...,    0,    0,    0],
        [-576,    0,    0, ...,    0,    0,    0],
        ...,
        [-567,    0,    0, ...,    0,    0,    0],
        [-558,    0,    0, ...,    0,    0,    0],
        [-558,    0,    0, ...,    0,    0,    0]],

       [[-585,    0,    0, ...,    0,    0,    0],
        [-576,    0,    0, ...,    0,    0,    0],
        [-576,    0,    0, ...,    0,    0,    0],
        ...,
        [-558,    0,    0, ...,    0,    0,    0],
        [-549,    0,    0, ...,    0,    0,    0],
        [-549,    0,    0, ...,    0,    0,    0]],

       [[-576,    0,    0, ...,    0,    0,    0],
        [-567,    0,    0, ...,    0,    0,    0],
        [-567,    0,    0, ...,    0,    0,    0],
        ...,
        [-549,    0,    0, ...,    0,    0,    0],
        [-549,    0,    0, ...,    0,    0,    0],
        [-540,    0,    0, ...,    0,    0,    0]],

       ...,

       [[-558,    0,    

array([[[-567,    0,    0, ...,    0,    0,    0],
        [-558,    0,    0, ...,    0,    0,    0],
        [-549,    0,    0, ...,    0,    0,    0],
        ...,
        [-549,    0,    0, ...,    0,    0,    0],
        [-540,    0,    0, ...,    0,    0,    0],
        [-540,    0,    0, ...,    0,    0,    0]],

       [[-567,    0,    0, ...,    0,    0,    0],
        [-558,    0,    0, ...,    0,    0,    0],
        [-558,    0,    0, ...,    0,    0,    0],
        ...,
        [-549,    0,    0, ...,    0,    0,    0],
        [-540,    0,    0, ...,    0,    0,    0],
        [-540,    0,    0, ...,    0,    0,    0]],

       [[-567,    0,    0, ...,    0,    0,    0],
        [-567,    0,    0, ...,    0,    0,    0],
        [-558,   -6,    0, ...,    0,    0,    0],
        ...,
        [-549,    0,    0, ...,    0,    0,    0],
        [-549,    0,    0, ...,    0,    0,    0],
        [-549,    0,    0, ...,    0,    0,    0]],

       ...,

       [[-585,    0,    

array([[[-585,    0,    0, ...,    0,    0,    0],
        [-585,    0,    0, ...,    0,    0,    0],
        [-576,    0,    0, ...,    0,    0,    0],
        ...,
        [-576,    0,    0, ...,    0,    0,    0],
        [-567,    0,    0, ...,    0,    0,    0],
        [-567,    0,    0, ...,    0,    0,    0]],

       [[-585,    0,    0, ...,    0,    0,    0],
        [-585,    0,    0, ...,    0,    0,    0],
        [-576,    0,    0, ...,    0,    0,    0],
        ...,
        [-567,    0,    0, ...,    0,    0,    0],
        [-567,    0,    0, ...,    0,    0,    0],
        [-567,    0,    0, ...,    0,    0,    0]],

       [[-594,    0,    0, ...,    0,    0,    0],
        [-594,    0,    0, ...,    0,    0,    0],
        [-585,   -6,    0, ...,    0,    0,    0],
        ...,
        [-576,    0,    0, ...,    0,    0,    0],
        [-567,    0,    0, ...,    0,    0,    0],
        [-558,   -6,    0, ...,    0,    0,    0]],

       ...,

       [[-603,    0,    

...

array([[[315,   6,   0, ...,   0,   0,   0],
        [306,   0,   0, ...,   0,   0,   0],
        [297,   6,   0, ...,   0,   0,   0],
        ...,
        [279,   0,   0, ...,   0,   0,   0],
        [279,   0,   0, ...,   0,   0,   0],
        [279,   0,   0, ...,   0,   0,   0]],

       [[315,   0,   0, ...,   0,   0,   0],
        [315,   0,   0, ...,   0,   0,   0],
        [306,   0,   0, ...,   0,   0,   0],
        ...,
        [297,   0,   0, ...,   0,   0,   0],
        [297,   0,   0, ...,   0,   0,   0],
        [288,   6,   0, ...,   0,   0,   0]],

       [[315,   0,   0, ...,   0,   0,   0],
        [315,   0,   0, ...,   0,   0,   0],
        [315,   0,   0, ...,   0,   0,   0],
        ...,
        [306,   0,   0, ...,   0,   0,   0],
        [306,   0,   0, ...,   0,   0,   0],
        [297,   6,   0, ...,   0,   0,   0]],

       ...,

       [[315,   0,   0, ...,   0,   0,   0],
        [324,   0,   0, ...,   0,   0,   0],
        [324,   0,   0, ...,   0,   0,   0

array([[[ 72, -30,  24, ...,   0,   0,   0],
        [387,  12,   0, ...,   0,   0,   0],
        [369,   0,   0, ...,   0,   0,   0],
        ...,
        [342,   0,   0, ...,   0,   0,   0],
        [333,   0,   0, ...,   0,   0,   0],
        [324,   0,   0, ...,   0,   0,   0]],

       [[126, -48,  24, ...,   0,   0,   0],
        [387,   6,   0, ...,   0,   0,   0],
        [369,   0,   0, ...,   0,   0,   0],
        ...,
        [351,   0,   0, ...,   0,   0,   0],
        [342,   0,   0, ...,   0,   0,   0],
        [324,   0,   0, ...,   0,   0,   0]],

       [[270, -36,  12, ...,   0,   0,   0],
        [360,   0,   0, ...,   0,   0,   0],
        [369,   0,   0, ...,   0,   0,   0],
        ...,
        [351,   0,   0, ...,   0,   0,   0],
        [342,   6,   0, ...,   0,   0,   0],
        [315,   0,   0, ...,   0,   0,   0]],

       ...,

       [[333, -24, -12, ...,   0,   0,   0],
        [369,   0,   0, ...,   0,   0,   0],
        [369,   0,   0, ...,   0,   0,   0

array([[[ 333,   96,   54, ...,    0,    0,    0],
        [ 360,  -12,   -6, ...,    0,    0,    0],
        [ 369,    0,    0, ...,    0,    0,    0],
        ...,
        [ 342,    6,    0, ...,    0,    0,    0],
        [ 324,    0,    0, ...,    0,    0,    0],
        [ 315,    0,    0, ...,    0,    0,    0]],

       [[ 378,  114,  -24, ...,    0,    0,    0],
        [ 342,  -24,  -12, ...,    0,    0,    0],
        [ 360,    0,    0, ...,    0,    0,    0],
        ...,
        [ 324,    0,    0, ...,    0,    0,    0],
        [ 315,    0,    0, ...,    0,    0,    0],
        [ 306,    6,    0, ...,    0,    0,    0]],

       [[ 342,   66,  -78, ...,    0,    0,    0],
        [ 306,  -48,  -12, ...,    0,    0,    0],
        [ 351,    0,    0, ...,    0,    0,    0],
        ...,
        [-225,  -12,    0, ...,    0,    0,    0],
        [-180,  -12,    0, ...,    0,    0,    0],
        [-126,  -12,    0, ...,    0,    0,    0]],

       ...,

       [[ 117,   54,  -9

array([[[ 297,   96, -114, ...,    0,    0,    0],
        [ 144,  -60,    0, ...,    0,    0,    0],
        [-774,  204,  168, ...,    0,    0,    0],
        ...,
        [-936,    0,    0, ...,    0,    0,    0],
        [-567, -258,  -18, ...,    0,    0,    0],
        [-342,   42,    6, ...,    0,    0,    0]],

       [[ 126,  168,   36, ...,    0,    0,    0],
        [ 108,  -66,    0, ...,    0,    0,    0],
        [-774,  186,  162, ...,    0,    0,    0],
        ...,
        [-900,    6,  -12, ...,    0,    0,    0],
        [-747,    6,    6, ...,    0,    0,    0],
        [-756,   42,   24, ...,    0,    0,    0]],

       [[  54,   60,  -42, ...,    0,    0,    0],
        [  72,  -72,    0, ...,    0,    0,    0],
        [-756,   48,   18, ...,    0,    0,    0],
        ...,
        [-900,   18,   18, ...,    0,    0,    0],
        [-126, -270, -144, ...,    0,    0,    0],
        [-801,    6,   18, ...,    0,    0,    0]],

       ...,

       [[-234,  -24,    

array([[[-684,   90,    0, ...,    0,    0,    0],
        [-738,  -12,  -36, ...,    0,    0,    0],
        [-441, -168,  -48, ...,    0,    0,    0],
        ...,
        [-252,  144,    0, ...,    0,    0,    0],
        [-423,   66,  -36, ...,    0,    0,    0],
        [-180,  -84,  -30, ...,    0,    0,    0]],

       [[-270,   36,   -6, ...,    0,    0,    0],
        [-342,    0,    0, ...,    0,    0,    0],
        [-369,   48,    0, ...,    0,    0,    0],
        ...,
        [-756,  -18,    6, ...,    0,    0,    0],
        [-711,    6,  -12, ...,    0,    0,    0],
        [-693,  -24,   12, ...,    0,    0,    0]],

       [[-207,   42,    6, ...,    0,    0,    0],
        [-279,   12,   -6, ...,    0,    0,    0],
        [-306,   12,    0, ...,    0,    0,    0],
        ...,
        [-414,   12,    0, ...,    0,    0,    0],
        [-459,   24,   -6, ...,    0,    0,    0],
        [-612,   84,   12, ...,    0,    0,    0]],

       ...,

       [[ -27,    6,    